In [ ]:
import mlflow

In [ ]:
mlflow.start_run(run_name="child_ana1", nested=True)

In [ ]:
num_samples = 3
interval = 33.0e-3
num_frames = 100
min_sigma = 1
max_sigma = 4
threshold = 50.0
overlap = 0.5

In [ ]:
nproc = 8

In [ ]:
import numpy
timepoints = numpy.linspace(0, interval * num_frames, num_frames + 1)

In [ ]:
import pathlib
inputpath = pathlib.Path("./artifacts")
artifacts = pathlib.Path("./artifacts")

In [ ]:
import scopyon

In [ ]:
from mlflow import log_metric, log_param, log_artifacts
log_artifacts("./artifacts")

In [ ]:
import warnings
warnings.simplefilter('ignore', RuntimeWarning)

for i in range(num_samples):
    imgs = [scopyon.Image(data) for data in numpy.load(inputpath / f"images{i:03d}.npy")]
    spots = [
        scopyon.analysis.spot_detection(
            img.as_array(), processes=nproc,
            min_sigma=min_sigma, max_sigma=max_sigma, threshold=threshold, overlap=overlap)
        for img in imgs]

    spots_ = []
    for t, data in zip(timepoints, spots):
        spots_.extend(([t] + list(row) for row in data))
    spots_ = numpy.array(spots_)
    numpy.save(artifacts / f"spots{i:03d}.npy", spots_)
    
    print("{} spots are detected in {} frames.".format(len(spots_), len(imgs)))

warnings.resetwarnings()

In [ ]:
mlflow.end_run()